In [1]:
class TestnetAddress:
    mainnet_url = "https://mainnet.infura.io/v3/f406ecf174e14211925011aac8d55cf6"
    goerli_url = "https://goerli.infura.io/v3/f406ecf174e14211925011aac8d55cf6"
    sepolia_url = "https://goerli.infura.io/v3/f406ecf174e14211925011aac8d55cf6"
    
    ganache_url = "http://127.0.0.1:7545"

In [6]:
from web3 import Web3
# TASK 1 (иногда подключается почему-то со второго раза, если в первый раз вылетел false - повторить попытку нужно)
print('''
Ethereum testnet:
1. mainnet
2. goerli
3. sepolia
4. ganache
'''
)
testnet_number = input("Enter: ")

if testnet_number == '1':
    testnet_url = TestnetAddress.mainnet_url
    
elif testnet_number == '2':
    testnet_url = TestnetAddress.goerli_url
    
elif testnet_number == '3':
    testnet_url = TestnetAddress.sepolia_url

elif testnet_number == '4':
    testnet_url = TestnetAddress.ganache_url
    

web3_test = Web3(Web3.HTTPProvider(testnet_url))
print("Connected to Testnet:",web3_test.isConnected())


Ethereum testnet:
1. mainnet
2. goerli
3. sepolia
4. ganache

Enter: 4
Connected to Testnet: True


In [ ]:
#-----

In [7]:
# TASK 2
# Получил монеты из крана https://goerli-faucet.pk910.de 
print("Transaction link: https://goerli.etherscan.io/tx/0x983d454e1c3475a7c8ccf477955a0b99c0e2d25ecad001fc93d58c75f5aad4cd")

Transaction link: https://goerli.etherscan.io/tx/0x983d454e1c3475a7c8ccf477955a0b99c0e2d25ecad001fc93d58c75f5aad4cd


In [8]:
# -----

In [9]:
# TASK 3
web3 = Web3(Web3.HTTPProvider(TestnetAddress.goerli_url))
print("Connected to goerli:",web3.isConnected())

Connected to goerli: True


In [10]:
from solcx import install_solc
install_solc(version='latest')
from solcx import compile_source

In [11]:
class LocalAccount:
    def __init__(self, web3, private_key, public_key):
        self.web3 = web3
        self.private_key = private_key
        self.public_key = public_key
    
    def get_balance_in_wei(self):
        return web3.eth.getBalance(self.public_key)
    
    def get_balance_in_ethers(self):
        return self.get_balance_in_wei()/10**18
    
    def transfer(self, to_account_public_key, ether_value):
        nonce = web3.eth.getTransactionCount(self.public_key)
        tx = {
            'nonce': nonce,
            'to': to_account_public_key,
            'value': web3.toWei(ether_value, 'ether'),
            'gas': 200000,
            'gasPrice': web3.toWei('50', 'gwei')
        }
        signed_tx = web3.eth.account.sign_transaction(tx, self.private_key)
        tx_hash = web3.eth.sendRawTransaction(signed_tx.rawTransaction)
        return web3.toHex(tx_hash)
    
    def deploy_contract(self, string_contract):
        compiled_sol = compile_source(
            string_contract,
            output_values = ['abi', 'bin']
        )
        contract_id, contract_interface = compiled_sol.popitem()
        bytecode = contract_interface['bin']
        abi = contract_interface['abi']
        
        
        contract = web3.eth.contract(abi=abi, bytecode=bytecode)
        nonce = web3.eth.getTransactionCount(self.public_key)
        
        tx = contract.constructor().buildTransaction({
            'from': self.public_key,
            'nonce': nonce
           # 'gas': 6721975, # from truffle docs
            #'gasPrice': 100000000000 # from truffle docs
        })
        
        signed_tx = web3.eth.account.sign_transaction(tx, self.private_key)
        tx_hash = web3.eth.sendRawTransaction(signed_tx.rawTransaction)
        
        tx_receipt = web3.eth.waitForTransactionReceipt(tx_hash)
        print("Contract deployed at address: {}".format(tx_receipt.contractAddress))
        
        deployed_contract = web3.eth.contract(
            address = tx_receipt.contractAddress,
            abi = abi
        )
        
        return deployed_contract, web3.toHex(tx_hash)
        


In [1]:
account_1_private = "-"
account_1_public = "-"

account_2_private = "-"
account_2_public = "-"

In [ ]:
### pre-requisite
account_1 = LocalAccount(web3, account_1_private, account_1_public)
account_2 = LocalAccount(web3, account_2_private, account_2_public)
print(account_1.public_key)
print(account_2.public_key)

In [14]:
class Task3BlockchainSolver:
    def __init__(self, web3, account):
        self.web3 = web3
        self.account = account
        
    def get_block_height(self):
        # в задании было написано at any time, поэтому предполагаю, что нужно все же последний блок брать
        # т.к. по идее любой height конкретного блока неизменный(в случае, если не будет успешной атаке)
        
        # в ином случае случае можно использовать 
        # web3.eth.get_block(hash/number)
        return self.web3.eth.block_number
    
    def get_info_about_last_block(self):
        return web3.eth.get_block('latest')
    
    def get_balance_in_wei(self):
        return self.account.get_balance_in_wei()
    
    def get_balance_in_ethers(self):
        return self.account.get_balance_in_ethers()

In [15]:
task3_solver = Task3BlockchainSolver(web3, account_1)
print(task3_solver.get_block_height())
print(task3_solver.get_info_about_last_block())
print(f"{task3_solver.get_balance_in_wei()} wei")
print(f"{task3_solver.get_balance_in_ethers()} ether")

7803846
AttributeDict({'baseFeePerGas': 8950684226, 'difficulty': 0, 'extraData': HexBytes('0x'), 'gasLimit': 30000000, 'gasUsed': 14325884, 'hash': HexBytes('0xb96eb5dffbbe69bbeb44810ea3232057c5f6f61e1e3946e9d270e642af08b60b'), 'logsBloom': HexBytes('0x10300909d86520251a851031a1d0060040a9434928011032408130090e284616014d00250201020453300d187484eb614aca098b0a520ac58000428f0126255001435068843d0089ce442b8a0e088666c003c4492d7ef10240b42a0e52904150433510881a0080953160b286180309509c00645440515d40012a8130850820cb0680b2248ca56105095a88039599417a3c7d15a22a88310810842964532144c1e6c52c11c40fde9602c1a8382248360a21b42880120e0331400a1042436ca0a02a69de220291a900741819800c16688e9908002e848362311a498d22c800662018b33509486202104850602981312e1208cc502b1148c1501371230212d02021'), 'miner': '0xc6e2459991BfE27cca6d86722F35da23A1E4Cb97', 'mixHash': HexBytes('0x594ae64d08f18ad2c81d6a397fdb011ce54abbed476c5cc0f1edb5d9915c1d62'), 'nonce': HexBytes('0x0000000000000000'), 'number': 7803846, 'parentHash': HexBytes('

0.31059009644790475 ether


In [ ]:
# -----

In [17]:
# TASK 4 https://goerli.etherscan.io/tx/0x8b5c13dd1083d4c08eec204129c18e60784549725ff71f9cc35429ad4c0a470c
print(f"tx hash: {account_1.transfer(account_2.public_key, 0.00007)}")

tx hash: 0x8b5c13dd1083d4c08eec204129c18e60784549725ff71f9cc35429ad4c0a470c


In [18]:
# -------

In [19]:
# Task 5
# смарт контракт с функцией сложения
task_5_contract = '''
// SPDX-License-Identifier: MIT

pragma solidity ^0.8.7;

contract SummatorContract {
    function add(int a, int b) public pure returns(int) {
        return a + b;
    }
}
'''

task5_deployed_contract, task5_tx_hash = account_1.deploy_contract(task_5_contract)

Contract deployed at address: 0x4A3B087bF420034decC488Ce69eeC45a24EC3C57


In [21]:
task5_result = task5_deployed_contract.functions.add(10, 7).call()
print(f"Contract creation transaction id {task5_tx_hash}")
print(f"Function add(10, 7) result: {task5_result}")
print(f"contract address: {task5_deployed_contract.address}")
print("URL https://goerli.etherscan.io/address/0x4A3B087bF420034decC488Ce69eeC45a24EC3C57")

Contract creation transaction id 0xc2123ccff20fd0a76b9762833840c517cb8eeb14a59ef8a0c6460d6c905fa187
Function add(10, 7) result: 17
contract address: 0x4A3B087bF420034decC488Ce69eeC45a24EC3C57
URL https://goerli.etherscan.io/address/0x4A3B087bF420034decC488Ce69eeC45a24EC3C57


In [ ]:
# -------

In [24]:
# Task 6
# contract code
'''
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.7;

import "@openzeppelin/contracts/token/ERC20/ERC20.sol";

contract DevToken is ERC20{
    constructor() ERC20("YKToken", "YKT"){
        _mint(msg.sender,100*10**18);
    }
}
'''
print(f"contract creation transaction id: 0xbd3f81379dba36a7852d343b276971dc1c3026f878bb0b34899a7b53b7e4e96f")
print(f"contract address: 0xa580697c849e1899ae15f6887CBD6Ba1f1BB7444")
print(f"letters YKT")
print(f"send 10 tokens transaction 0x9eab226942617ff032bd716002c31cb45f2ad1c4e4b852e9a2bf580ae586b89c")
print(f"send 10 tokens transaction link https://goerli.etherscan.io/tx/0x9eab226942617ff032bd716002c31cb45f2ad1c4e4b852e9a2bf580ae586b89c")

contract creation transaction id: 0xbd3f81379dba36a7852d343b276971dc1c3026f878bb0b34899a7b53b7e4e96f
contract address: 0xa580697c849e1899ae15f6887CBD6Ba1f1BB7444
letters YKT
send 10 tokens transaction 0x9eab226942617ff032bd716002c31cb45f2ad1c4e4b852e9a2bf580ae586b89c
send 10 tokens transaction link https://goerli.etherscan.io/tx/0x9eab226942617ff032bd716002c31cb45f2ad1c4e4b852e9a2bf580ae586b89c
